<a href="https://colab.research.google.com/github/Sjing2/SJ_project/blob/master/%5B%EB%A7%88%EC%BC%80%ED%8C%85%20%ED%94%84%EB%A1%9C%EC%A0%9D%ED%8A%B8%5D%20%EC%8B%9C%EC%9E%A5%20%EB%B6%84%EC%84%9D%EC%9D%84%20%EC%9C%84%ED%95%9C%20%ED%85%8D%EC%8A%A4%ED%8A%B8%20%EB%B0%8F%20%EC%BB%A4%EB%AE%A4%EB%8B%88%ED%8B%B0%20%EB%A7%88%EC%9D%B4%EB%8B%9D%20%EA%B8%B0%EB%B0%98%20%EC%9D%B8%EC%8A%A4%ED%83%80%EA%B7%B8%EB%9E%A8%20%EC%82%AC%EC%9A%A9%EC%9E%90%20%ED%94%84%EB%A1%9C%ED%8C%8C%EC%9D%BC%EB%A7%81/2.%20Preprocessing/13.%5B%EC%A0%84%EC%B2%98%EB%A6%AC%5D%20ID%20Region.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive/


In [ ]:
df = pd.read_csv('/content/gdrive/My Drive/인스타그램_프로파일링/Post/Final/cc_post.csv')

In [ ]:
words = ['여행', '출장', '본가', '휴가', '호캉스', '바캉스', '투어', '펜션', '호텔', '숙박', '공항', '픽업', '해수욕장', 'MT', '엠티', '객실']

In [ ]:
# content 내 words 포함된 단어의 index
idx_content = []

for i, v in enumerate(df['content']):
    for w in words:
        if w in str(v):
            idx_content.append(i)

In [ ]:
# hashtag 내 words 포함된 단어의 index
idx_hashtag = []

for i, v in enumerate(df['hashtag']):
    for w in words:
        if w in str(v):
            idx_hashtag.append(i)

In [ ]:
# content와 hashtag에 words가 중복으로 있는 index 제거 후 new_df로 저장
idx = []
idx.extend(idx_content)
idx.extend(idx_hashtag)
idx = list(set(idx))
print('words가 포함된 post 수: {}'.format(len(idx)))

words가 포함된 post 수: 8519


In [ ]:
new_df = df.drop(idx)
new_df.reset_index(drop=True, inplace=True)

In [ ]:
print('기존 post수: {}, 제거 후 post 수: {}'.format(len(df), len(new_df)))

기존 post수: 110588, 제거 후 post 수: 102069


In [ ]:
# 모든 content가 nan값인 user 제거

In [ ]:
unique_id = np.unique(new_df['insta_id'])

In [ ]:
content_id_delete = []
for i in unique_id:
    length = len(new_df[new_df['insta_id']==i])
    cnt = 0
    for j in new_df[new_df['insta_id']==i]['content']:
        try:
            np.isnan(j)
            cnt += 1
        except:
            pass
    if cnt == length:
        content_id_delete.append(i)

In [ ]:
new_new_df = new_df
for i in content_id_delete:
    new_new_df = new_new_df[new_new_df['insta_id']!=i]
new_new_df.reset_index(drop=True, inplace=True)

In [ ]:
print('모든 content가 nan값인 유저 수: {}'.format(len(content_id_delete)))

모든 content가 nan값인 유저 수: 6


In [ ]:
# 각 id의 region_tag추출. nan값은 제거.

In [ ]:
id_list = np.unique(new_new_df['insta_id'])

In [ ]:
region = []

for i in id_list:
    region_tag = []
    for j in new_new_df[new_new_df['insta_id']==i]['region_tag']:
        try:
            np.isnan(j)
        except:
            region_tag.append(j)
    region.append(region_tag)

In [ ]:
id_region = dict(zip(id_list, region))

In [ ]:
id_delete = []

for i in id_region:
    if len(id_region[i]) == 0:
        id_delete.append(i)

In [ ]:
for i in id_delete:
    id_region.pop(i)

In [ ]:
# 저장
blank = pd.DataFrame(['' for i in range(len(id_region.keys()))], columns = ['region'])
ids = pd.DataFrame(list(id_region.keys()), columns = ['insta_id'])
regions = pd.DataFrame(list(id_region.values()))

In [ ]:
final = pd.concat([blank, ids, regions], axis = 1)

In [ ]:
final.to_csv('/content/gdrive/My Drive/프로젝트/cc_post_final/cc_id_region2.csv', index=False)